In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf
from kaggle.competitions import nflrush
import time
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt



In [70]:
env = nflrush.make_env()

Exception: You can only call `make_env()` once.

In [423]:
data = pd.read_csv('train.csv', low_memory = False)


In [424]:
data.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [425]:
data.shape[0]

509762

In [450]:
#split into training and labels
train = data.drop(['Yards'], axis=1).copy()
label = data['Yards'].copy()

In [451]:
train.isna().sum().sort_values(ascending=False)

WindDirection             80234
WindSpeed                 67430
Temperature               48532
GameWeather               43648
StadiumType               32934
FieldPosition              6424
Humidity                   6160
OffenseFormation            110
DefendersInTheBox            66
Orientation                  18
Dir                          14
Season                        0
Down                          0
PossessionTeam                0
GameClock                     0
Quarter                       0
YardLine                      0
PlayId                        0
Team                          0
DisplayName                   0
NflId                         0
Distance                      0
Dis                           0
A                             0
S                             0
Y                             0
X                             0
JerseyNumber                  0
NflIdRusher                   0
HomeScoreBeforePlay           0
PlayerCollegeName             0
Turf    

In [452]:
#train['WindSpeed'].value_counts()

In [453]:
#TODO
# clean up missing data
# preprocess features and standardize them
# encode categorical features
#https://www.kaggle.com/shahules/how-about-some-nn-keras-starter

In [454]:
#WindDirection
# This function has been updated to reflect what Subin An (https://www.kaggle.com/subinium) mentioned in comments below.
# WindDirection is indicated by the direction that wind is flowing FROM - https://en.wikipedia.org/wiki/Wind_direction

def clean_wind_direction(wind_direction):
    wd = str(wind_direction).upper()
    if wd == 'N' or 'FROM N' in wd:
        return 'north'
    if wd == 'S' or 'FROM S' in wd:
        return 'south'
    if wd == 'W' or 'FROM W' in wd:
        return 'west'
    if wd == 'E' or 'FROM E' in wd:
        return 'east'
    
    if 'FROM SW' in wd or 'FROM SSW' in wd or 'FROM WSW' in wd:
        return 'south west'
    if 'FROM SE' in wd or 'FROM SSE' in wd or 'FROM ESE' in wd:
        return 'south east'
    if 'FROM NW' in wd or 'FROM NNW' in wd or 'FROM WNW' in wd:
        return 'north west'
    if 'FROM NE' in wd or 'FROM NNE' in wd or 'FROM ENE' in wd:
        return 'north east'
    
    if 'NW' in wd or 'NORTHWEST' in wd:
        return 'north west'
    if 'NE' in wd or 'NORTH EAST' in wd:
        return 'north east'
    if 'SW' in wd or 'SOUTHWEST' in wd:
        return 'south west'
    if 'SE' in wd or 'SOUTHEAST' in wd:
        return 'south east'

    return 'none'

train['WindDirection'] = train['WindDirection'].apply(clean_wind_direction)

In [455]:
#Windspeed
def windspeed(x):
    x=str(x)
    if x.isdigit():
        return int(x)
    elif (x.isalpha()):
        return np.nan
    elif (x.isalnum()):
        return int(x.upper().split('M')[0])                             #return 12 incase of 12mp or 12 MPH
    elif '-' in x:
        return int((int(x.split('-')[0])+int(x.split('-')[1]))/2)   # return average windspeed incase of 11 - 20 etc..
    else:
        return np.nan
    

train['WindSpeed']=train['WindSpeed'].apply(windspeed)

train['WindSpeed'].fillna(train['WindSpeed'].median(),inplace=True)


In [456]:
train['Humidity'].value_counts()

0.0     45254
70.0    28908
56.0    13420
67.0    13288
42.0    12782
        ...  
24.0     1034
14.0      968
7.0       968
29.0      902
32.0      880
Name: Humidity, Length: 86, dtype: int64

In [457]:
# Humidity and Temperature
train['Humidity'].fillna(method='ffill', inplace=True)
#interpolate()
train['Humidity'].isna().sum()

0

In [460]:
train['Temperature'].fillna(method='ffill', inplace=True)

In [461]:
train.isna().sum().sort_values(ascending=False)

GameWeather               43648
StadiumType               32934
FieldPosition              6424
OffenseFormation            110
DefendersInTheBox            66
Orientation                  18
Dir                          14
JerseyNumber                  0
Distance                      0
Down                          0
PossessionTeam                0
GameClock                     0
Quarter                       0
YardLine                      0
Season                        0
WindDirection                 0
DisplayName                   0
HomeScoreBeforePlay           0
Dis                           0
A                             0
S                             0
Y                             0
X                             0
Team                          0
PlayId                        0
NflId                         0
NflIdRusher                   0
VisitorScoreBeforePlay        0
Position                      0
Humidity                      0
Temperature                   0
Turf    

In [14]:
# Orientation, Direction, Defenders, Offense
print(np.nanmean(train['Orientation'].values))
na_map = {
    'Orientation': train['Orientation'].mean(),
    'Dir': train['Dir'].mean(),
    'DefendersInTheBox': (train['DefendersInTheBox'].median()),
    'OffenseFormation': 'blank'
}

train.fillna(na_map, inplace=True)
train['DefendersInTheBox'].isna().sum()

180.24777914011733


0

In [15]:
#Field Position, undefined when yard line = 50
print(train['FieldPosition'].isna().sum())
train['FieldPosition'] = np.where(train['YardLine'] == 50, train['PossessionTeam'], train['FieldPosition'])
print(train['FieldPosition'].isna().sum())

6424
0


In [16]:
#Game weather
def group_game_weather(weather):
    rain = [
        'Rainy', 'Rain Chance 40%', 'Showers',
        'Cloudy with periods of rain, thunder possible. Winds shifting to WNW, 10-20 mph.',
        'Scattered Showers', 'Cloudy, Rain', 'Rain shower', 'Light Rain', 'Rain'
    ]
    overcast = [
        'Cloudy, light snow accumulating 1-3"', 'Party Cloudy', 'Cloudy, chance of rain',
        'Coudy', 'Cloudy, 50% change of rain', 'Rain likely, temps in low 40s.',
        'Cloudy and cold', 'Cloudy, fog started developing in 2nd quarter',
        'Partly Clouidy', '30% Chance of Rain', 'Mostly Coudy', 'Cloudy and Cool',
        'cloudy', 'Partly cloudy', 'Overcast', 'Hazy', 'Mostly cloudy', 'Mostly Cloudy',
        'Partly Cloudy', 'Cloudy'
    ]
    clear = [
        'Partly clear', 'Sunny and clear', 'Sun & clouds', 'Clear and Sunny',
        'Sunny and cold', 'Sunny Skies', 'Clear and Cool', 'Clear and sunny',
        'Sunny, highs to upper 80s', 'Mostly Sunny Skies', 'Cold',
        'Clear and warm', 'Sunny and warm', 'Clear and cold', 'Mostly sunny',
        'T: 51; H: 55; W: NW 10 mph', 'Clear Skies', 'Clear skies', 'Partly sunny',
        'Fair', 'Partly Sunny', 'Mostly Sunny', 'Clear', 'Sunny'
    ]
    snow  = ['Heavy lake effect snow', 'Snow']
    indoor  = ['N/A Indoor', 'Indoors', 'Indoor', 'N/A (Indoors)', 'Controlled Climate']
    
    if weather in rain:
        return 'rain'
    elif weather in overcast:
        return 'overcast'
    elif weather in clear:
        return 'clear'
    elif weather in snow:
        return 'snow'
    elif weather in indoor:
        return 'indoor'
    
    return 'unspecified'

train['GameWeather'] = train['GameWeather'].apply(group_game_weather)


In [17]:
#Stadiums
def group_stadium_types(stadium):
    outdoor  = [
        'Outdoor', 'Outdoors', 'Cloudy', 'Heinz Field', 
        'Outdor', 'Ourdoor', 'Outside', 'Outddors', 
        'Outdoor Retr Roof-Open', 'Oudoor', 'Bowl'
    ]
    indoor_closed = [
        'Indoors', 'Indoor', 'Indoor, Roof Closed', 'Indoor, Roof Closed', 
        'Retractable Roof', 'Retr. Roof-Closed', 'Retr. Roof - Closed', 'Retr. Roof Closed',
    ]
    indoor_open   = ['Indoor, Open Roof', 'Open', 'Retr. Roof-Open', 'Retr. Roof - Open']
    dome_closed   = ['Dome', 'Domed, closed', 'Closed Dome', 'Domed', 'Dome, closed']
    dome_open     = ['Domed, Open', 'Domed, open']
    
    if stadium in outdoor:
        return 'outdoor'
    elif stadium in indoor_closed:
        return 'indoor closed'
    elif stadium in indoor_open:
        return 'indoor open'
    elif stadium in dome_closed:
        return 'dome closed'
    elif stadium in dome_open:
        return 'dome open'
    else:
        return 'unknown'
    
train['StadiumType'] = train['StadiumType'].apply(group_stadium_types)

In [18]:
#BirthDate, GameHour and Time
import datetime
train['TimeHandoff'] = train['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeSnap'] = train['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
train['TimeDelta'] = train.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)

#train['BirthYear']=train['PlayerBirthDate'].apply(lambda x : int(x.split('/')[2]))
def strtoseconds(txt):
    txt = txt.split(':')
    ans = int(txt[0])*60 + int(txt[1]) + int(txt[2])/60
    return ans
train['GameClock']=train['GameClock'].apply(strtoseconds)


In [19]:
train['PlayerBirthDate'] = train['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

seconds_in_year = 60*60*24*365.25
train['PlayerAge'] = train.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
train = train.drop(['TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)

In [21]:
#Height
#train['PlayerHeight'] = z
train['PlayerHeight'].value_counts()
train['PlayerHeight']=train['PlayerHeight'].apply(lambda x : 30*int(x.split('-')[0]) + 3*int(x.split('-')[1]))

In [22]:
#DefensePersonnel
def process_defense(x):
    num=[]
    num=x.split(',')
    dl=int(num[0].split(' ')[0])
    lb=int(num[1].split(' ')[1])
    db=int(num[2].split(' ')[1])
    if(len(num)>3):
         ol=int(num[3].split(' ')[1])
    else:
         ol=0
    return [dl,lb,db,ol]

values=train['DefensePersonnel'].apply(process_defense)
u,v,x,y=list(map(list,zip(*values)))




In [23]:
train['DL']=u
train['LB']=v
train['BL']=x
train['OL']=y
train.drop(['DefensePersonnel'],axis=1,inplace=True)

In [24]:
train['PlayDirection'] = train['PlayDirection'].apply(lambda x: x.strip() == 'right')

train['Team'] = train['Team'].apply(lambda x: x.strip()=='home')

In [25]:
train['X'] = train.apply(lambda row: row['X'] if row['PlayDirection'] else 120-row['X'], axis=1)
#from https://www.kaggle.com/scirpus/hybrid-gp-and-nn
def new_orientation(angle, play_direction):
    if play_direction == 0:
        new_angle = 360.0 - angle
        if new_angle == 360.0:
            new_angle = 0.0
        return new_angle
    else:
        return angle
    
train['Orientation'] = train.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
train['Dir'] = train.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)

In [26]:
#Remove irrelevant features
train = train.drop(['GameId','PlayId','NflId','NflIdRusher'], axis=1)


In [27]:
#Additional features
train['PlayerBMI'] = (0.5*train['PlayerWeight']/(0.01*train['PlayerHeight'])**2)

In [28]:
def get_cats(train):
    cat_features = []
    for col in train.columns:
        if train[col].dtype =='object':
            cat_features.append(col)
    #print(cat_features)
    return cat_features

In [29]:
train['WindDirection'].value_counts()

none          150634
south west     77110
north west     68288
north east     64306
south east     52998
south          32978
north          25080
west           24860
east           13508
Name: WindDirection, dtype: int64

In [30]:
# encoding categorical values

from sklearn.preprocessing import LabelEncoder

cats = get_cats(train)

lbdic={}
for c in cats:
    lb=LabelEncoder()
    lb=lb.fit(train[c].values)
    lbdic[c]=lb
    train[c]=lb.transform(train[c].values)

In [31]:
train.head()

,Team,X,Y,S,A,Dis,Orientation,Dir,DisplayName,JerseyNumber,...,Humidity,WindSpeed,WindDirection,TimeDelta,PlayerAge,DL,LB,BL,OL,PlayerBMI
0,False,46.09,34.84,1.69,1.13,0.40,278.01,182.82,775,29,...,77.0,8.0,7,1.0,28.692760,2,3,6,0,32.716049
1,False,45.33,32.64,0.42,1.35,0.01,332.39,161.30,71,97,...,77.0,8.0,7,1.0,28.457305,2,3,6,0,40.312421
2,False,46.00,33.20,1.22,0.59,0.31,356.99,157.27,1228,50,...,77.0,8.0,7,1.0,28.629790,2,3,6,0,37.792895
3,False,48.54,27.70,0.42,0.54,0.02,0.23,254.36,664,56,...,77.0,8.0,7,1.0,34.795430,2,3,6,0,34.293553
4,False,50.68,35.42,1.82,2.43,0.16,347.37,195.69,1814,38,...,77.0,8.0,7,1.0,30.061685,2,3,6,0,31.790123


In [462]:
#TODO: Refactor all pipeline steps as function so Test can be processed the same way.

def pipeline(df, train=False):
    #print(df.head())
    df['WindDirection'] = df['WindDirection'].apply(clean_wind_direction)
    df['WindSpeed']=df['WindSpeed'].apply(windspeed)

    df['WindSpeed'].fillna(df['WindSpeed'].median(),inplace=True)
    
    df['Humidity'].fillna(method='ffill', inplace=True)
    df['Temperature'].fillna(method='ffill', inplace=True)
    df.fillna(na_map, inplace=True)
    df['FieldPosition'] = np.where(df['YardLine'] == 50, df['PossessionTeam'], df['FieldPosition'])
    df['GameWeather'] = df['GameWeather'].apply(group_game_weather)
    df['StadiumType'] = df['StadiumType'].apply(group_stadium_types)
    
    
    
    df['TimeHandoff'] = df['TimeHandoff'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeSnap'] = df['TimeSnap'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))
    df['TimeDelta'] = df.apply(lambda row: (row['TimeHandoff'] - row['TimeSnap']).total_seconds(), axis=1)
    df['GameClock']=df['GameClock'].apply(strtoseconds)
    df['PlayerBirthDate'] = df['PlayerBirthDate'].apply(lambda x: datetime.datetime.strptime(x, "%m/%d/%Y"))

    seconds_in_year = 60*60*24*365.25
    df['PlayerAge'] = df.apply(lambda row: (row['TimeHandoff']-row['PlayerBirthDate']).total_seconds()/seconds_in_year, axis=1)
    df = df.drop(['TimeHandoff', 'TimeSnap', 'PlayerBirthDate'], axis=1)
    df['PlayerHeight']=df['PlayerHeight'].apply(lambda x : 30*int(x.split('-')[0]) + 3*int(x.split('-')[1]))

    values=df['DefensePersonnel'].apply(process_defense)
    u,v,x,y=list(map(list,zip(*values)))
    df['DL']=u
    df['LB']=v
    df['BL']=x
    df['OL']=y
    df.drop(['DefensePersonnel'],axis=1,inplace=True)

    df['PlayDirection'] = df['PlayDirection'].apply(lambda x: x.strip() == 'right')

    df['Team'] = df['Team'].apply(lambda x: x.strip()=='home')

    df['X'] = df.apply(lambda row: row['X'] if row['PlayDirection'] else 120-row['X'], axis=1)

    df['Orientation'] = df.apply(lambda row: new_orientation(row['Orientation'], row['PlayDirection']), axis=1)
    df['Dir'] = df.apply(lambda row: new_orientation(row['Dir'], row['PlayDirection']), axis=1)

    #Remove irrelevant features
    df = df.drop(['GameId','PlayId','NflId','NflIdRusher'], axis=1)
    #Additional features
    df['PlayerBMI'] = (0.5*df['PlayerWeight']/(0.01*df['PlayerHeight'])**2)

    #Encode categorical features
    cats = get_cats(df)

    lbdic={}
    for c in cats:
        lb=LabelEncoder()
        lb=lb.fit(df[c].values)
        lbdic[c]=lb
        df[c]=lb.transform(df[c].values)
        
    #print(df.head)
    if(train):
        features = df.drop(['Yards'], axis=1).copy()
        label = df['Yards'].copy()
        return features, label
    else:
        return features


In [463]:
data = pd.read_csv('train.csv', low_memory = False)
features, label = pipeline(data, train=True)

In [464]:
features.isna().sum().sort_values(ascending=False)

PlayerBMI                 0
YardLine                  0
OffenseFormation          0
VisitorScoreBeforePlay    0
HomeScoreBeforePlay       0
FieldPosition             0
Distance                  0
Down                      0
PossessionTeam            0
GameClock                 0
Quarter                   0
Season                    0
DefendersInTheBox         0
JerseyNumber              0
DisplayName               0
Dir                       0
Orientation               0
Dis                       0
A                         0
S                         0
Y                         0
X                         0
OffensePersonnel          0
PlayDirection             0
OL                        0
GameWeather               0
BL                        0
LB                        0
DL                        0
PlayerAge                 0
TimeDelta                 0
WindDirection             0
WindSpeed                 0
Humidity                  0
Temperature               0
Turf                

In [465]:
#turns yardage into categorical format to match softmax output. We will aggregate the NN PDF into a CDF afterwards.

def transform_label(label):
    return tf.keras.utils.to_categorical(label, 199)

#Turns yardage gained into cumulative function: 000..111111. Flips to 1 at the index of the "yards gained"
#do not use 
def IGNOREtransform_label(y):
    Y_train=np.zeros((y.shape[0],199))
    for i,yard in enumerate(y):
        Y_train[i, yard+99:] = np.ones(shape=(1, 100-yard))
    
    return Y_train

Y = transform_label(label)

from sklearn.preprocessing import StandardScaler

#optional scaling of X
#scaler = StandardScaler()
#X = scaler.fit_transform(features)

In [466]:
trans_label = transform_label(label)

In [467]:
#X data, shape
print(features.head())
print(features.shape)
print(trans_label.shape)
print(trans_label)


    Team      X      Y     S     A   Dis  Orientation     Dir  DisplayName  \
0  False  46.09  34.84  1.69  1.13  0.40       278.01  182.82          775   
1  False  45.33  32.64  0.42  1.35  0.01       332.39  161.30           71   
2  False  46.00  33.20  1.22  0.59  0.31       356.99  157.27         1228   
3  False  48.54  27.70  0.42  0.54  0.02         0.23  254.36          664   
4  False  50.68  35.42  1.82  2.43  0.16       347.37  195.69         1814   

   JerseyNumber  ...  Humidity  WindSpeed  WindDirection  TimeDelta  \
0            29  ...      77.0        8.0              7        1.0   
1            97  ...      77.0        8.0              7        1.0   
2            50  ...      77.0        8.0              7        1.0   
3            56  ...      77.0        8.0              7        1.0   
4            38  ...      77.0        8.0              7        1.0   

   PlayerAge  DL  LB  BL  OL  PlayerBMI  
0  28.692760   2   3   6   0  32.716049  
1  28.457305   2   3

In [468]:
from sklearn import preprocessing

x = features.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
print(x_scaled)
print(x_scaled[0].shape)
X = pd.DataFrame(x_scaled)
print(X)

/home/floriansuri/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


[[0.         0.36951282 0.60421245 ... 0.71428571 0.         0.28888889]
 [0.         0.36297125 0.56391941 ... 0.71428571 0.         0.56235828]
 [0.         0.36873816 0.57417582 ... 0.71428571 0.         0.47165533]
 ...
 [1.         0.72387674 0.46153846 ... 0.42857143 0.         0.63537147]
 [1.         0.70072302 0.41245421 ... 0.42857143 0.         0.58034894]
 [1.         0.66827337 0.44871795 ... 0.42857143 0.         0.30433874]]
(47,)
         0         1         2         3         4         5         6   \
0       0.0  0.369513  0.604212  0.179596  0.076818  0.287770  0.772250   
1       0.0  0.362971  0.563919  0.044633  0.091774  0.007194  0.923306   
2       0.0  0.368738  0.574176  0.129649  0.040109  0.223022  0.991639   
3       0.0  0.390601  0.473443  0.044633  0.036710  0.014388  0.000639   
4       0.0  0.409020  0.614835  0.193411  0.165194  0.115108  0.964917   
...     ...       ...       ...       ...       ...       ...       ...   
509757  1.0  0.719659  0.

In [469]:
## Normalize value to [0, 1]
print(features.index.values)
X = X.iloc[X.index.values]
X = X.as_matrix()
print(X)

[     0      1      2 ... 509759 509760 509761]
[[0.         0.36951282 0.60421245 ... 0.71428571 0.         0.28888889]
 [0.         0.36297125 0.56391941 ... 0.71428571 0.         0.56235828]
 [0.         0.36873816 0.57417582 ... 0.71428571 0.         0.47165533]
 ...
 [1.         0.72387674 0.46153846 ... 0.42857143 0.         0.63537147]
 [1.         0.70072302 0.41245421 ... 0.42857143 0.         0.58034894]
 [1.         0.66827337 0.44871795 ... 0.42857143 0.         0.30433874]]


/home/floriansuri/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [470]:
X

array([[0.        , 0.36951282, 0.60421245, ..., 0.71428571, 0.        ,
        0.28888889],
       [0.        , 0.36297125, 0.56391941, ..., 0.71428571, 0.        ,
        0.56235828],
       [0.        , 0.36873816, 0.57417582, ..., 0.71428571, 0.        ,
        0.47165533],
       ...,
       [1.        , 0.72387674, 0.46153846, ..., 0.42857143, 0.        ,
        0.63537147],
       [1.        , 0.70072302, 0.41245421, ..., 0.42857143, 0.        ,
        0.58034894],
       [1.        , 0.66827337, 0.44871795, ..., 0.42857143, 0.        ,
        0.30433874]])

In [ ]:
#split into custom validation/test set - or can we use their test set as black box? 

In [481]:
# baseline model Copy from Hw2 - tune accordingly

#redefine an optimizer that inherits from SGD or any optimizer but introduces synchronization
def optim_SGD(lr = 0.001, mom=0.99):
    #Declare Optimizer
    optim = tf.keras.optimizers.SGD(learning_rate = lr, momentum = mom) #, clipnorm=1.)
    return optim

#set up layers for baseline model: remove convolutions and set dense layers... Softmax output with 199 dim
def new_model(optim, d1= 32, d2 =32, d3 =32):

    models = tf.keras.models
    layers = tf.keras.layers
    #Instantiate an empty model
    model = models.Sequential()

    
    model.add(layers.Dense(d1, activation="relu", input_shape=(47,))) #[features.shape[1]]))
    model.add(layers.Flatten())
    model.add(layers.Dense(d2, activation="relu"))
    #model.add(layers.Flatten())
    model.add(layers.Dense(d3, activation="relu"))

    #Output Layer with softmax activation
    #model.add(layers.Flatten())
    model.add(layers.Dense(199, activation="softmax"))
    #model.add(layers.Flatten())
    # Compile the model
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=optim, metrics=["accuracy"])

    print(model.summary())

    return model

In [482]:
optim = optim_SGD(lr = 0.01, mom=0)



In [384]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import six
import copy
import numpy as np
from six.moves import zip

import tensorflow.keras.backend as K

class SGD(tf.keras.optimizers.SGD):

    def __init__(self, learning_rate=0.01, momentum=0.,
                 nesterov=False, **kwargs):
        super(SGD, self).__init__(learning_rate, momentum, nesterov, **kwargs)

    #interfaces.legacy_get_updates_support
    #@K.symbolic
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        print(self)
        lr = self.learning_rate
        #if self.initial_decay > 0:
        #    lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
        #                                              K.dtype(self.decay))))
        # momentum
        shapes = [K.int_shape(p) for p in params]
        moments = [K.zeros(shape, name='moment_' + str(i))
                   for (i, shape) in enumerate(shapes)]
        self.weights = [self.iterations] + moments
        for p, g, m in zip(params, grads, moments):
            v = self.momentum * m - lr * g  # velocity
            self.updates.append(K.update(m, v))

            if self.nesterov:
                new_p = p + self.momentum * v - lr * g
            else:
                new_p = p + v

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            print(p)
            self.updates.append(K.update(p, new_p))
        return self.updates

o = SGD(learning_rate=0.0)

In [483]:
model = new_model(optim)

Model: "sequential_86"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_322 (Dense)            (None, 32)                1536      
_________________________________________________________________
flatten_69 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_323 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_324 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_325 (Dense)            (None, 199)               6567      
Total params: 10,215
Trainable params: 10,215
Non-trainable params: 0
_________________________________________________________________
None


In [288]:
#print(model.get_weights())
a = model.predict(X.iloc[[2]]).flatten()
b = trans_label[2]
#print(a, b)
#print(a*b)


AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [484]:
print(X.shape, trans_label.shape)
#x_train = X.reshape(X.shape[0], 47)

Z =  np.zeros((509762, 47))
print(X)
t_x = np.array(X)
t_y = np.array(trans_label)

(509762, 47) (509762, 199)
[[0.         0.36951282 0.60421245 ... 0.71428571 0.         0.28888889]
 [0.         0.36297125 0.56391941 ... 0.71428571 0.         0.56235828]
 [0.         0.36873816 0.57417582 ... 0.71428571 0.         0.47165533]
 ...
 [1.         0.72387674 0.46153846 ... 0.42857143 0.         0.63537147]
 [1.         0.70072302 0.41245421 ... 0.42857143 0.         0.58034894]
 [1.         0.66827337 0.44871795 ... 0.42857143 0.         0.30433874]]


In [ ]:
model.fit(x=t_x,y=t_y, epochs=10, validation_split = 0.1, verbose=1)

Train on 458785 samples, validate on 50977 samples
Epoch 1/10
218816/458785 [=============>................] - ETA: 10s - loss: 2.9680 - acc: 0.1208

In [405]:
import math

'''
for i in range (math.ceil(math.log(t_x.shape[0]) / math.log(2))):
    x_t = np.array(t_x[0:(1 << i)])
    y_t = np.array(t_y[0:(1 << i)])
    print(x_t.shape)
    hist = model.fit(x=x_t,y=y_t, epochs=1, validation_split = 0, verbose=0)
    
    print(i, hist.history.get("loss"))
''' 

model = new_model(optim)
n = 1 << 13 + 1 << 12
hist = model.fit(x=t_x[0:n],y=t_y[0:n], epochs=1, validation_split = 0, verbose=0)
print(n, hist.history.get("loss"))
#,  batch_size=32, steps_per_epoch = 1000)

Model: "sequential_82"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_306 (Dense)            (None, 32)                1536      
_________________________________________________________________
flatten_65 (Flatten)         (None, 32)                0         
_________________________________________________________________
dense_307 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_308 (Dense)            (None, 32)                1056      
_________________________________________________________________
dense_309 (Dense)            (None, 199)               6567      
Total params: 10,215
Trainable params: 10,215
Non-trainable params: 0
_________________________________________________________________
None
67108864 [nan]


In [476]:
print(model.get_weights())

[array([[-0.25861433,  0.15810229, -0.27280802, ...,  0.15857446,
        -0.2688716 ,  0.10894391],
       [ 0.0694183 , -0.03973237, -0.12177023, ..., -0.16553348,
        -0.05753512,  0.04791745],
       [ 0.02021869, -0.23066446,  0.17310008, ...,  0.22438858,
         0.08881959,  0.2685546 ],
       ...,
       [-0.08214569,  0.1138791 ,  0.2307817 , ...,  0.07269611,
        -0.23558825, -0.16619784],
       [-0.09838273, -0.1839545 ,  0.09741232, ..., -0.2623272 ,
         0.27068678,  0.04588333],
       [ 0.20320407,  0.259509  ,  0.07787147, ..., -0.1611301 ,
         0.0406305 ,  0.22180235]], dtype=float32), array([-4.29120200e-06, -1.39994445e-05,  1.80825097e-07, -5.76440616e-05,
        1.86014160e-09,  9.03424461e-06, -6.35120432e-06,  8.34754758e-07,
       -4.13965827e-05,  1.17061054e-05,  4.06908657e-06,  3.21457492e-06,
       -3.43648344e-06, -2.72177800e-07, -3.41822306e-05, -2.07748853e-05,
       -6.59798479e-06,  1.28029633e-05, -3.50473711e-05, -1.99605984e

In [479]:
x_test= np.ones(([47,]))
print(x_scaled[2].shape)
#print(features.iloc[[2]])
#print(model.predict(x_scaled[2]))
#print(model.predict(X.iloc[[2]]))
print(X[0])
a = np.array([X[0]])
print(a.shape)
print(model.predict(a))
#print(False*6)

(47,)
[0.         0.36951282 0.60421245 0.17959617 0.07681849 0.28776978
 0.77225    0.50783333 0.34768955 0.28571429 0.         0.69387755
 0.         0.94888889 0.64516129 0.66666667 0.02564103 0.64516129
 0.         0.         0.625      0.16363636 0.5        0.
 0.30769231 0.25991189 0.82       0.875      0.64516129 0.48387097
 0.         0.31481481 0.38983051 0.8        0.21052632 0.
 0.61363636 0.77       0.33333333 0.875      0.14285714 0.39592364
 0.28571429 0.5        0.71428571 0.         0.28888889]
(1, 47)
[[0.00494009 0.00480225 0.00520356 0.00492679 0.00583737 0.00439809
  0.00457874 0.00557206 0.00461957 0.00462194 0.00461632 0.00525921
  0.00447101 0.00456975 0.00561349 0.00591571 0.00457629 0.00473909
  0.00593363 0.00437634 0.00523422 0.00496195 0.00544863 0.00503398
  0.0049375  0.0045382  0.00449494 0.00468009 0.00504894 0.00448047
  0.00596542 0.00503889 0.00494658 0.00503336 0.0048166  0.00447492
  0.00549605 0.00555923 0.00490524 0.00502339 0.00494457 0.00471415


In [42]:
#copy function that evaluates loss




Exception: You can only call `make_env()` once.

In [44]:
for (testdf, samplepredictiondf) in env.iter_test():
    print(testdf.head(1)) 
    
    #predictionsdf = makemypredictions(testdf, samplepredictiondf) 
    #env.predict(predictionsdf)

#for test, sample in tqdm.tqdm(env.iter_test()):
#    make_prediction(test,sample,env,model,df_train)
    
#env.write_submission_file()

Exception: You can only iterate over `iter_test()` once.

In [47]:
df = pipeline(testdf)
print(df.head(1))

TypeError: strptime() argument 1 must be str, not Timestamp